### Merging Meteorological Data (90 Stations)

In [3]:
# Here we will merge all meteorological data into a one file and also change the structure of the data into a right format so we can analyze further
# Please unzip all meteorological data and load them directly next to this code
# Load relevant libaries and data
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

# List of region names corresponding to the CSV files
stations = ["burlington", "columbus junct 2 ssw", "fairfield", "iowa - southeast climate division", "keosauqua", "burlingtonairport", "donnellson", "fortmadison", "keokuk", "mount-pleasant", 
           "allerton", "centerville", "chariton", "des moines area", "iowa - south central climate division", "iowa drought region 5 - southeast", "newton", "osceola-3-wsw", "oskaloosa", "rathbun dam",
           "atlantic-1-ne", "clarinda", "corning", "creston", "greenfield", "iowa - southwest climate division", "iowa drought region 4 - southwest", "oakland-2-e", "shenandoah-1-ne", "winterset",
           "iowadroughtregion1", "iowanorthwestclimate", "lemars", "primghar", "rockrapids", "sibley5", "siouxcenter", "spencer", "siouxrapids", "spencerarea",
           "algona", "estherville4e", "esthervillemunicipal", "fortdodge", "iowadroughtregion2", "jefferson", "pocahontas", "saccity", "emmetsburg", "webstercity",
           "audubon", "battle creek 3ne", "carroll-2-ssw", "denison", "ida grove area", "iowa - west central climate division", "little sioux 2nw", "mapleton no. 2", "sac-city", "sioux city area",
           "elkader6ssw", "fayette", "iowadroughtregion3", "iowanortheastclimatedivision", "newhampton", "oelwein1e", "oelweinarea", "postville", "stanley", "waukon",
           "ames 5 se", "ames area", "ames municipal ap", "ames-8-wsw", "boone", "grinnell-3-sw", "grundy-center", "iowa average", "marshalltown municipal ap", "toledo",
           "anamosa3ssw", "cascade", "cedarrapids1", "cedarrapidsarea", "clinton1", "dubuquearea", "iowaeastcentral", "lowden", "manchester2", "maquioketa2w"] 
#Southeast, SouthCentral,Southwest, Northwest, Northcentral,  Westcentral, Northeast, Central, eastcentral

# Iterate through each region
for region in stations:
    try:
        # Load the CSV for the current region (no header in the file)
        df = pd.read_csv(f'{region}.csv', header=None)
        
        # Add 'Region' column with the region name for this dataset
        df['Stations'] = region
        
        # Add proper column names: ['Type', 'Year', 'Jan', 'Feb', ..., 'Dec', 'Region']
        columns = ['Type', 'Year'] + [f'Month_{i}' for i in range(1, 13)] + ['Stations']
        df.columns = columns
        
        # Melt the DataFrame to transform the data structure as required
        df_melted = df.melt(id_vars=["Type", "Year", "Stations"], var_name="Month", value_name="Value")
        
        # Pivot the melted DataFrame to get 'prec', 'tmin', and 'tmax' as columns and each month as rows
        df_pivoted = df_melted.pivot_table(index=["Year", "Month", "Stations"], columns="Type", values="Value").reset_index()
        
        # Sort the DataFrame by Year and Month to get the desired order of rows from Month_1 of each year to Month_12 of each year
        df_pivoted = df_pivoted.sort_values(by=["Year", "Month"])
        
        # Save the modified dataframe back to a CSV (overwrite the original file or save with a new name)
        df_pivoted.to_csv(f'{region}_modified.csv', index=False)
        
        # Log the success
        print(f"Successfully modified data for region: {region}")
        print(df_pivoted.head())  # Show the first 5 rows of the modified file
        
    except FileNotFoundError:
        print(f"File for region {region} not found.")
    except Exception as e:
        print(f"An error occurred for region {region}: {e}")

Successfully modified data for region: burlington
Type  Year     Month    Stations    prec   tmax   tmin
0     2015   Month_1  burlington   30.48   0.43  -9.37
1     2015  Month_10  burlington   56.14  19.02   6.70
2     2015  Month_11  burlington  157.48  12.78   2.02
3     2015  Month_12  burlington  123.71   7.67  -0.20
4     2015   Month_2  burlington   39.12  -3.00 -13.67
Successfully modified data for region: columbus junct 2 ssw
Type  Year     Month              Stations    prec   tmax   tmin
0     2015   Month_1  columbus junct 2 ssw   22.35  -0.66 -10.32
1     2015  Month_10  columbus junct 2 ssw   52.07  18.39   6.11
2     2015  Month_11  columbus junct 2 ssw  123.19  11.98   1.06
3     2015  Month_12  columbus junct 2 ssw  117.61   6.83  -0.91
4     2015   Month_2  columbus junct 2 ssw   37.09  -3.58 -14.28
Successfully modified data for region: fairfield
Type  Year     Month   Stations    prec   tmax   tmin
0     2015   Month_1  fairfield   22.36  -0.07 -10.50
1     2015  M

In [4]:
# Try to merge data into a single file starting from regional meteological data.

# Initialize an empty list to store DataFrames
df_list = []

# Loop through each region and read the corresponding CSV file
for region in stations:
    df = pd.read_csv(f"{region}_modified.csv")
    df['Stations'] = region  # Add a 'Region' column
    df_list.append(df)

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(df_list)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv("merged_regions.csv", index=False)


In [5]:
# We found out that the merged_regions.csv does not have region column so here we will add region column using for loop. 
# Region column is necessary for us to carry out inner-join task with the explanatory dataset (Corn.csv) at the later stage.

# Load the merged file
df = pd.read_csv("merged_regions.csv")

# List of stations
stations = ["burlington", "columbus junct 2 ssw", "fairfield", "iowa - southeast climate division", "keosauqua", "burlingtonairport", "donnellson", "fortmadison", "keokuk", "mount-pleasant", 
           "allerton", "centerville", "chariton", "des moines area", "iowa - south central climate division", "iowa drought region 5 - southeast", "newton", "osceola-3-wsw", "oskaloosa", "rathbun dam",
           "atlantic-1-ne", "clarinda", "corning", "creston", "greenfield", "iowa - southwest climate division", "iowa drought region 4 - southwest", "oakland-2-e", "shenandoah-1-ne", "winterset",
           "iowadroughtregion1", "iowanorthwestclimate", "lemars", "primghar", "rockrapids", "sibley5", "siouxcenter", "spencer", "siouxrapids", "spencerarea",
           "algona", "estherville4e", "esthervillemunicipal", "fortdodge", "iowadroughtregion2", "jefferson", "pocahontas", "saccity", "emmetsburg", "webstercity",
           "audubon", "battle creek 3ne", "carroll-2-ssw", "denison", "ida grove area", "iowa - west central climate division", "little sioux 2nw", "mapleton no. 2", "sac-city", "sioux city area",
           "elkader6ssw", "fayette", "iowadroughtregion3", "iowanortheastclimatedivision", "newhampton", "oelwein1e", "oelweinarea", "postville", "stanley", "waukon",
           "ames 5 se", "ames area", "ames municipal ap", "ames-8-wsw", "boone", "grinnell-3-sw", "grundy-center", "iowa average", "marshalltown municipal ap", "toledo",
           "anamosa3ssw", "cascade", "cedarrapids1", "cedarrapidsarea", "clinton1", "dubuquearea", "iowaeastcentral", "lowden", "manchester2", "maquioketa2w"]

# Initialize regions
regions = ["southeast", "southcentral", "southwest", "northwest", "northcentral", "westcentral", "northeast", "central", "eastcentral"]

# Create a dictionary to map stations to regions
station_to_region = {}
for i, region in enumerate(regions):
    for station in stations[i*10:(i+1)*10]:
        station_to_region[station] = region

# Add a new column 'Region' to the DataFrame
df['Region'] = df['Stations'].map(station_to_region)

# Save the modified DataFrame to a new CSV file
df.to_csv("merged_regions_modified.csv", index=False)

print("The new column 'Region' has been added and the modified DataFrame has been saved to 'merged_regions_modified.csv'.")


The new column 'Region' has been added and the modified DataFrame has been saved to 'merged_regions_modified.csv'.


In [6]:
# Checking whether the code executed according to our expectation
df['Region'].value_counts()

Region
southeast       1200
southcentral    1200
southwest       1200
northwest       1200
northcentral    1200
westcentral     1200
northeast       1200
central         1200
eastcentral     1200
Name: count, dtype: int64

### Merging Corn + Merged_Meteorological Data (90 Stations)
Now, it is better to delete all meteorological data 90 stations + 90 Modified files and only leave "Merged_regions_modified.csv"

And please add "Corn.csv" file on the platform

In [8]:
## Here we will merge merged_region_modified file with the corn data (Explanatory variable).
## Please unzip corn folder and load the file directly next to this code like you did for meteorological data
## Now we will merge the corn data with the merged regions data
## We will first load the corn data, aggregate it, and then merge it with the merged regions data

# Load the CSV files
corn_df = pd.read_csv('Corn.csv')

# Keep only Year, Ag District, and Value columns in the Corn data and create a copy to avoid warnings
corn_df_reduced = corn_df[['Year', 'Ag District', 'Value']].copy()
 
# Convert the 'Value' column to numeric, handling commas
corn_df_reduced['Value'] = corn_df_reduced['Value'].replace(',', '', regex=True).astype(float)

# Aggregate by Year and Ag District, summing the values
corn_df_aggregated = corn_df_reduced.groupby(['Year', 'Ag District']).sum().reset_index()

# Renaming 'Ag District' to 'Region' in the first dataset to match the column name in merged_regions_df
corn_df_aggregated.rename(columns={'Ag District': 'Region'}, inplace=True)

corn_df_aggregated


,Year,Region,Value
0,2015,CENTRAL,369962000.0
1,2015,EAST CENTRAL,257582000.0
2,2015,NORTH CENTRAL,364763000.0
3,2015,NORTHEAST,311052000.0
4,2015,NORTHWEST,382845000.0
...,...,...,...
67,2022,NORTHWEST,337170000.0
68,2022,SOUTH CENTRAL,74502000.0
69,2022,SOUTHEAST,152549000.0
70,2022,SOUTHWEST,174484000.0


In [11]:
# Let's check how we are going to merge these two csv using regions
# First check how regions are spelled in each dataset

# Merged_regions_df has a column and we need to see what unique values are in that column
merged_regions_df['Region'].unique()

array(['southeast', 'southcentral', 'southwest', 'northwest',
       'northcentral', 'westcentral', 'northeast', 'central',
       'eastcentral'], dtype=object)

In [12]:
# Same for corn_df_aggregated 
corn_df_aggregated['Region'].unique()

array(['CENTRAL', 'EAST CENTRAL', 'NORTH CENTRAL', 'NORTHEAST',
       'NORTHWEST', 'SOUTH CENTRAL', 'SOUTHEAST', 'SOUTHWEST',
       'WEST CENTRAL'], dtype=object)

In [13]:
# Create a mapping dictionary to standardize the values in the 'Region' column
# This is a manual process, but it is necessary to ensure that the values in the 'Region' column are consistent across both datasets
standardization_map = { 'CENTRAL': 'central', 'EAST CENTRAL': 'eastcentral', 'NORTH CENTRAL': 'northcentral', 'NORTHEAST': 'northeast', 'NORTHWEST': 'northwest', 'SOUTH CENTRAL': 'southcentral', 'SOUTHEAST': 'southeast', 'SOUTHWEST': 'southwest', 'WEST CENTRAL': 'westcentral' }

# Standardize region names in the DataFrame to lowercase and remove spaces 
corn_df_aggregated['Region'] = corn_df_aggregated['Region'].str.lower().str.replace(' ', '') 
# Perform replacement using the standardized map 
corn_df_aggregated['Region'] = corn_df_aggregated['Region'].replace(standardization_map)


corn_df_aggregated['Region'].unique()
merged_regions_df['Region'].unique()


# Merge the dataframes on 'Year' and 'Region'
merged_df = pd.merge(corn_df_aggregated, merged_regions_df, on=['Year', 'Region'], how='inner')

# Save it to a new csv file
merged_df.to_csv('corn_merged.csv', index=False)

# We have successfully merged two dataset into one under the file name "corn_merged.csv"

In [14]:
# For a better analysis, we are changing Month to numeric values

# Load the corn_merged.csv file
df = pd.read_csv("corn_merged.csv")

# Convert the 'Month' column values to numeric
df['Month'] = df['Month'].str.replace('Month_', '').astype(int)

# Save the modified DataFrame to a new CSV file
df.to_csv("corn_merged_modified.csv", index=False)

print("The 'Month' column values have been converted to numeric and the modified DataFrame has been saved to 'corn_merged_modified.csv'.")

The 'Month' column values have been converted to numeric and the modified DataFrame has been saved to 'corn_merged_modified.csv'.


### Merging Corn_Merged with Biodiesel Data

And please add "US_Bioenergy_Statistics.csv" file on the platform

In [18]:
# Load the dataset without headers
df = pd.read_csv('US_Bioenergy_Statistics.csv', header=None)
 
# Because it does not have a column name, let's add column names
df.columns = ['Year', 'Month', 'Biodiesel', 'Diesel', 'Price Difference']
 
# Create a dictionary to map month names to numeric values
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}
 
# Replace month names with numeric values
df['Month'] = df['Month'].map(month_map)
 
# Save the modified DataFrame to a new CSV file with metadata
df.to_csv('US_Bioenergy_Statistics_with_metadata.csv', index=False)
 
# Display the first few rows of the modified DataFrame to verify
print(df.head())

   Year  Month  Biodiesel  Diesel  Price Difference
0  2014      1       3.28    3.89             -0.61
1  2014      2       3.37    3.98             -0.61
2  2014      3       3.70    4.00             -0.30
3  2014      4       3.76    3.96             -0.20
4  2014      5       3.75    3.94             -0.19


In [19]:
# Load the datasets
corn_data = pd.read_csv('corn_merged_modified.csv')
bioenergy_data = pd.read_csv('US_Bioenergy_Statistics_with_metadata.csv')
 
# Merge the datasets on the 'Year' column
merged_data = pd.merge(corn_data, bioenergy_data, on='Year')
 
# Display the first few rows of the merged dataset
print("First few rows of the merged dataset:")
print(merged_data.head())
 
# Save the merged dataset to a new CSV file
merged_data.to_csv('Corn&Biodieselmerged_dataset.csv', index=False)

merged_data

First few rows of the merged dataset:
   Year   Region        Value  Month_x   Stations   prec  tmax   tmin  \
0  2015  central  369962000.0        1  ames 5 se  17.04 -0.79 -11.54   
1  2015  central  369962000.0        1  ames 5 se  17.04 -0.79 -11.54   
2  2015  central  369962000.0        1  ames 5 se  17.04 -0.79 -11.54   
3  2015  central  369962000.0        1  ames 5 se  17.04 -0.79 -11.54   
4  2015  central  369962000.0        1  ames 5 se  17.04 -0.79 -11.54   

   Month_y  Biodiesel  Diesel  Price Difference  
0        1       2.85    3.00             -0.15  
1        2       2.92    2.86              0.06  
2        3       2.89    2.90             -0.01  
3        4       2.92    2.78              0.13  
4        5       3.11    2.89              0.22  


,Year,Region,Value,Month_x,Stations,prec,tmax,tmin,Month_y,Biodiesel,Diesel,Price Difference
0,2015,central,369962000.0,1,ames 5 se,17.04,-0.79,-11.54,1,2.85,3.00,-0.15
1,2015,central,369962000.0,1,ames 5 se,17.04,-0.79,-11.54,2,2.92,2.86,0.06
2,2015,central,369962000.0,1,ames 5 se,17.04,-0.79,-11.54,3,2.89,2.90,-0.01
3,2015,central,369962000.0,1,ames 5 se,17.04,-0.79,-11.54,4,2.92,2.78,0.13
4,2015,central,369962000.0,1,ames 5 se,17.04,-0.79,-11.54,5,3.11,2.89,0.22
...,...,...,...,...,...,...,...,...,...,...,...,...
103675,2022,westcentral,372069000.0,9,sioux city area,19.31,27.76,10.89,8,6.71,5.01,1.70
103676,2022,westcentral,372069000.0,9,sioux city area,19.31,27.76,10.89,9,6.79,4.99,1.80
103677,2022,westcentral,372069000.0,9,sioux city area,19.31,27.76,10.89,10,6.81,5.21,1.60
103678,2022,westcentral,372069000.0,9,sioux city area,19.31,27.76,10.89,11,6.72,5.26,1.47


In [20]:
# Load the modified biodiesel dataset
df1 = pd.read_csv('US_Bioenergy_Statistics_with_metadata.csv')
 
# Load the corn_merged_modified dataset
df2 = pd.read_csv('corn_merged_modified.csv')
 
# Merge the datasets on 'Year' and 'Month'
merged_df = pd.merge(df1, df2, on=['Year', 'Month'])
 
# Display the first few rows of the merged DataFrame to verify
print(merged_df.head())
 
 
merged_df.to_csv('finished.csv', index=False)

   Year  Month  Biodiesel  Diesel  Price Difference   Region        Value  \
0  2015      1       2.85     3.0             -0.15  central  369962000.0   
1  2015      1       2.85     3.0             -0.15  central  369962000.0   
2  2015      1       2.85     3.0             -0.15  central  369962000.0   
3  2015      1       2.85     3.0             -0.15  central  369962000.0   
4  2015      1       2.85     3.0             -0.15  central  369962000.0   

            Stations   prec  tmax   tmin  
0          ames 5 se  17.04 -0.79 -11.54  
1          ames area  17.20 -0.79 -11.54  
2  ames municipal ap  28.97  0.29 -10.68  
3         ames-8-wsw   4.83  1.19  -9.76  
4              boone  16.27 -0.39 -11.72  
